# [Project Euler Problem 84](https://projecteuler.net/problem=84)

This solution uses a Markov chain approach. The matrix is $120\times120$: an entry for each square, repeated three times representing the current double count.

In [8]:
M = 40
p_go = 0
p_jail = 10
p_g2j = 30
p_cc1 = 2
p_cc2 = 17
p_cc3 = 33
p_ch1 = 7
p_ch2 = 22
p_ch3 = 36
p_c1 = 11
p_e3 = 24
p_h2 = 39
p_r1 = 5
p_r2 = 15
p_r3 = 25
p_r4 = 35
p_u1 = 12
p_u2 = 28

In [9]:
def p_next_r(p):
    if p < p_r1 or p >= p_r4:
        return p_r1
    if p < p_r2:
        return p_r2
    if p < p_r3:
        return p_r3
    return p_r4

def p_next_u(p):
    if p < p_u1 or p >= p_u2:
        return p_u1
    return p_u2

In [101]:
import numpy as np
def transition_matrix(dice_faces):
    mat = np.zeros([3*M, 3*M])
    for r1 in range(1, dice_faces+1):
        for r2 in range(1, dice_faces+1):
            roll = r1 + r2
            is_double = r1 == r2
            for double_count in range(3):
                for p in range(M):
                    i = M*double_count + p
                    if is_double:
                        if double_count == 2:
                            mat[p_jail,i] += 1
                            continue
                        else:
                            offset = M*(1+double_count)
                    else:
                        offset = 0
                    q = (p + roll) % M
                    if q == p_cc1 or q == p_cc2 or q == p_cc3:
                        mat[offset + p_go, i] += 1/16
                        mat[offset + p_jail, i] += 1/16
                        mat[offset + q, i] += 14/16
                    elif q == p_ch1 or q == p_ch2 or q == p_ch3:
                        mat[offset + p_go, i] += 1/16
                        mat[offset + p_jail, i] += 1/16
                        mat[offset + p_c1, i] += 1/16
                        mat[offset + p_e3, i] += 1/16
                        mat[offset + p_h2, i] += 1/16
                        mat[offset + p_r1, i] += 1/16
                        mat[offset + p_next_r(q), i] += 2/16
                        mat[offset + p_next_u(q), i] += 1/16
                        mat[offset + (q - 3) % M, i] += 1/16
                        mat[offset + q, i] += 6/16
                    elif q == p_g2j:
                        mat[offset + p_jail, i] += 1
                    else:
                        mat[offset + q, i] += 1
    return mat / np.sum(mat, axis=0)

In [120]:
def square_frequencies(dice_faces, eps=0.0001):
    mat = transition_matrix(dice_faces)
    v1 = np.zeros(3*M)
    v2 = np.zeros(3*M)
    v2[0] = 1
    while np.sum(np.abs(v1-v2)) > eps:
        v1 = v2
        v2 = mat.dot(v2)
    freqs = v2[0:M] + v2[M:2*M] + v2[2*M:]
    return sorted(zip(range(M), freqs), key= lambda x: x[1], reverse=True)

In [117]:
def answer_string(freqs):
    return ''.join(['00' if f[0]==0 else str(f[0]) for f in freqs[:3]])

In [121]:
freqs = square_frequencies(6)
print(freqs[:3])
print(answer_string(freqs))

[(10, 0.062330040682600445), (24, 0.03183238702092352), (0, 0.03088749568024003)]
102400


In [122]:
freqs = square_frequencies(4)
print(freqs[:3])
print(answer_string(freqs))

[(10, 0.07014033570368253), (15, 0.03614888098203577), (24, 0.0328680936657339)]
101524
